In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('energy2.csv')
df.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Undernourished,Recovery_rate_percentage,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,Afghanistan,0.0000,4.7774,0.8504,0.0,37.1186,0.1501,0.0000,1.4757,1.2006,...,45.2476,2.3012,0.7504,4.5,29.8,86.801012,NaN,NaN,NaN,NaN
1,Albania,0.9120,16.0930,1.0591,0.0,16.2107,0.8091,0.1471,3.8982,3.8688,...,33.9070,2.8244,2.7508,22.3,6.2,60.413005,NaN,NaN,NaN,NaN
2,Algeria,0.0896,6.0326,0.1941,0.0,25.0112,0.4181,0.1195,3.1805,1.2543,...,43.9749,5.7638,2.0457,26.6,3.9,68.425558,NaN,NaN,NaN,NaN
3,Angola,1.9388,4.6927,0.2644,0.0,18.3521,0.0441,0.8372,2.3133,2.9302,...,45.3184,4.2741,0.3525,6.8,25.0,92.089522,NaN,NaN,NaN,NaN
4,Antigua and Barbuda,2.3041,15.3672,1.5429,0.0,13.7215,0.2057,1.7280,3.6824,7.0356,...,34.6225,4.6904,1.2960,19.1,NaN,64.930556,NaN,NaN,NaN,NaN


In [3]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [4]:
df1=df.dropna()

In [5]:
df1.head()

,Country,Alcoholic Beverages,Animal Products,Animal fats,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,...,Stimulants,Sugar Crops,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Obesity,Undernourished,Recovery_rate_percentage
0,Afghanistan,0.0000,4.7774,0.8504,0.0,37.1186,0.1501,0.0000,1.4757,1.2006,...,0.0750,0.0,2.2261,0.1251,45.2476,2.3012,0.7504,4.5,29.8,86.801012
1,Albania,0.9120,16.0930,1.0591,0.0,16.2107,0.8091,0.1471,3.8982,3.8688,...,0.2501,0.0,3.4422,0.3972,33.9070,2.8244,2.7508,22.3,6.2,60.413005
2,Algeria,0.0896,6.0326,0.1941,0.0,25.0112,0.4181,0.1195,3.1805,1.2543,...,0.1493,0.0,3.9869,0.2240,43.9749,5.7638,2.0457,26.6,3.9,68.425558
3,Angola,1.9388,4.6927,0.2644,0.0,18.3521,0.0441,0.8372,2.3133,2.9302,...,0.0441,0.0,2.7539,0.0000,45.3184,4.2741,0.3525,6.8,25.0,92.089522
5,Argentina,1.4354,14.9869,1.0650,0.0,16.7927,0.8643,0.2006,1.4663,9.4459,...,0.2315,0.0,7.0536,0.0463,34.9900,5.5410,0.8643,28.5,4.6,89.647840


In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [7]:
label_encoder=preprocessing.LabelEncoder()
df1['Country']=label_encoder.fit_transform(df1['Country'])

<ipython-input-7-51c39540de4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Country']=label_encoder.fit_transform(df1['Country'])


In [8]:
y=df1.Recovery_rate_percentage
x=df1.drop('Recovery_rate_percentage',axis=1)
np.random.seed(42)
x_shadow=x.apply(np.random.permutation)
x_shadow.columns={'shadow_'+ feat for feat in x.columns}
x_boruta=pd.concat([x,x_shadow],axis=1)

In [11]:
from sklearn.ensemble import RandomForestRegressor
forest=RandomForestRegressor(max_depth=5,random_state=42)
forest.fit(x_boruta,y)
feat_imp_x=forest.feature_importances_[:len(x.columns)]
feat_imp_shadow=forest.feature_importances_[len(x.columns):]
hits=feat_imp_x>feat_imp_shadow.max()

In [14]:
hits = np.zeros((len(x.columns)))
### repeat 20 times
for iter_ in range(70):
   ### make X_shadow by randomly permuting each column of X
   np.random.seed(iter_)
   x_shadow = x.apply(np.random.permutation)
   x_boruta = pd.concat([x, x_shadow], axis = 1)
   ### fit a random forest (suggested max_depth between 3 and 7)
   forest = RandomForestRegressor(max_depth = 5, random_state = 42)
   forest.fit(x_boruta, y)
   ### store feature importance
   feat_imp_x = forest.feature_importances_[:len(x.columns)]
   feat_imp_shadow = forest.feature_importances_[len(x.columns):]
   ### compute hits for this trial and add to counter
   hits += (feat_imp_x > feat_imp_shadow.max())

In [15]:
print(hits)

[ 2.  1.  0.  0.  0. 31.  0.  0.  0. 30. 18.  1.  0.  0.  2.  0.  0.  0.
  0.  0.  0.  0.  5.  0.  0.  0.]
